In [2]:
import os
import numpy as np
import rasterio
import pickle
import skimage.feature

In [3]:

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


Loading Sentinal 1 Data

In [4]:
# Define the base directory for the data
base_dir = r'c:\\Users\\Dell\\Desktop\\Project downlaods'  # Update this to your actual directory path

# Paths to specific data files

mask_dir = os.path.join(base_dir, 'mask')
bbox_file = os.path.join(base_dir, 'bbox.pkl')
meta_info_file = os.path.join(base_dir, 'meta_info.pkl')
timestamp_file = os.path.join(base_dir, 'timestamp.pkl')

# Load polarization data
vv_path = os.path.join(base_dir, 'VV.npy')
vh_path = os.path.join(base_dir, 'VH.npy')
vv_data = np.load(vv_path)
vh_data = np.load(vh_path)

In [5]:
# Load mask data
# Assuming there's a specific mask file you need, update the file name accordingly
mask_path = os.path.join(base_dir, 'MASK.npy')  # Update 'MASK.npy' if needed
mask_data = np.load(mask_path)

# Load bounding box information
with open(bbox_file, 'rb') as f:
    bbox = pickle.load(f)

# Example of how to use this data
# Print basic information
print("VV Data Shape:", vv_data.shape)
print("VH Data Shape:", vh_data.shape)
print("Mask Data Shape:", mask_data.shape)
print("Bounding Box:", bbox)



VV Data Shape: (120, 2400, 2400, 1)
VH Data Shape: (120, 2400, 2400, 1)
Mask Data Shape: (120, 2400, 2400, 1)
Bounding Box: 408000.1106542662,5831999.778170622,432000.083625369,5856000.355934586


C:\Users\Dell\AppData\Local\Temp\ipykernel_872\4038265090.py:15: SHDeprecationWarning: The string representation of `BBox` will change to match its `repr` representation.
  print("Bounding Box:", bbox)


In [6]:
from abc import ABC, abstractmethod

class _FeatureDict(ABC):
    @abstractmethod
    def _parse_feature_value(self, value):
        """Implementation to parse feature values."""
        pass

# Implement the class with the required method
class ConcreteFeatureDict(_FeatureDict):
    def _parse_feature_value(self, value):
        # Actual implementation here
        return value


In [7]:
# Load timestamps
with open(timestamp_file, 'rb') as f:
    timestamps = pickle.load(f)
print("Timestamps:", timestamps)

Timestamps: [datetime.datetime(2019, 1, 4, 16, 51, 27, tzinfo=tzlocal()), datetime.datetime(2019, 1, 5, 16, 44, 4, tzinfo=tzlocal()), datetime.datetime(2019, 1, 10, 16, 52, 7, tzinfo=tzlocal()), datetime.datetime(2019, 1, 11, 16, 43, 22, tzinfo=tzlocal()), datetime.datetime(2019, 1, 16, 16, 51, 27, tzinfo=tzlocal()), datetime.datetime(2019, 1, 17, 16, 44, 4, tzinfo=tzlocal()), datetime.datetime(2019, 1, 22, 16, 52, 7, tzinfo=tzlocal()), datetime.datetime(2019, 1, 23, 16, 43, 22, tzinfo=tzlocal()), datetime.datetime(2019, 1, 28, 16, 51, 26, tzinfo=tzlocal()), datetime.datetime(2019, 2, 3, 16, 52, 7, tzinfo=tzlocal()), datetime.datetime(2019, 2, 4, 16, 43, 21, tzinfo=tzlocal()), datetime.datetime(2019, 2, 9, 16, 51, 26, tzinfo=tzlocal()), datetime.datetime(2019, 2, 10, 16, 44, 3, tzinfo=tzlocal()), datetime.datetime(2019, 2, 15, 16, 52, 6, tzinfo=tzlocal()), datetime.datetime(2019, 2, 16, 16, 43, 21, tzinfo=tzlocal()), datetime.datetime(2019, 2, 21, 16, 51, 26, tzinfo=tzlocal()), datetim

In [8]:
# Processing example: Masking the VV data
masked_vv_data = np.where(mask_data == 1, vv_data, np.nan)  # Assuming mask value '1' means valid data

# Save processed data (example)
output_path = os.path.join(base_dir, 'processed_VV_data.npy')
np.save(output_path, masked_vv_data)
print("Processed data saved at:", output_path)

Processed data saved at: c:\\Users\\Dell\\Desktop\\Project downlaods\processed_VV_data.npy


Loading Sentinal 2 Data
    

In [9]:
import os
import numpy as np
import pickle

# Correct file path
files_path = "c:\\Users\\Dell\\Desktop\\Sentinal 2\\"

# Check file sizes
file_sizes = {f: os.path.getsize(files_path + f) for f in os.listdir(files_path)}
print("File sizes (in bytes):", file_sizes)

# Using memory mapping to load large .npy file
bands = np.memmap(files_path + 'BANDS.npy', dtype='float32', mode='r')
clp = np.load(files_path + 'CLP.npy')
is_data = np.load(files_path + 'IS_DATA.npy')
norm_factors = np.load(files_path + 'NORM_FACTORS.npy')
scl = np.load(files_path + 'SCL.npy')

# Loading .pkl files with error handling
def load_pickle_file(file_path):
    try:
        with open(file_path, 'rb') as f:
            return pickle.load(f)
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

bbox = load_pickle_file(files_path + 'bbox.pkl')
meta_info = load_pickle_file(files_path + 'meta_info.pkl')
timestamp = load_pickle_file(files_path + 'timestamp.pkl')

# Exploring the data
print("Bands shape:", bands.shape)
print("CLP shape:", clp.shape)
print("IS_DATA shape:", is_data.shape)
print("Normalization factors shape:", norm_factors.shape)
print("SCL shape:", scl.shape)

print("Bounding box:", bbox)
print("Metadata:", meta_info)
print("Timestamp:", timestamp)


File sizes (in bytes): {'azcopy_windows_amd64_10.25.1.zip': 16538997, 'BANDS.npy': 19906560128, 'bbox.pkl': 184, 'CLP.npy': 829440128, 'Crops_GT_Brandenburg_Doc.pdf': 71678, 'IS_DATA.npy': 829440128, 'meta_info.pkl': 348, 'NORM_FACTORS.npy': 704, 'requirements (1).txt': 105, 'requirements.txt': 105, 's1dataset.py': 7580, 's2dataset.py': 7868, 'SCL.npy': 829440128, 'sitemap.xml': 524, 'timestamp.pkl': 14759, 'visualization.ipynb': 595176, '__pycache__': 0}
Error loading c:\Users\Dell\Desktop\Sentinal 2\meta_info.pkl: Can't instantiate abstract class _FeatureDict with abstract method _parse_feature_value
Bands shape: (4976640032,)
CLP shape: (144, 2400, 2400, 1)
IS_DATA shape: (144, 2400, 2400, 1)
Normalization factors shape: (144, 1)
SCL shape: (144, 2400, 2400, 1)
Bounding box: 408000.1106542662,5831999.778170622,432000.083625369,5856000.355934586
Metadata: None
Timestamp: [datetime.datetime(2019, 1, 2, 10, 26, 3, tzinfo=tzlocal()), datetime.datetime(2019, 1, 4, 10, 16, 3, tzinfo=tzloc

C:\Users\Dell\AppData\Local\Temp\ipykernel_872\687672365.py:39: SHDeprecationWarning: The string representation of `BBox` will change to match its `repr` representation.
  print("Bounding box:", bbox)


bands is a 1D array with 4,976,640,032 elements.
CLP, IS_DATA, and SCL are 4D arrays with shape (144, 2400, 2400, 1).
norm_factors is a 2D array with shape (144, 1).

'BANDS.npy' file as before.
print the shape of the bands array to see its actual dimensions.
print the first 10 elements of the array to get an idea of its contents.

In [10]:
import numpy as np
import os

# Correct file path
files_path = "c:\\Users\\Dell\\Desktop\\Sentinal 2\\"

# Load the data
bands = np.memmap(os.path.join(files_path, 'BANDS.npy'), dtype='float32', mode='r')

# Print the shape of the bands array
print("Bands shape:", bands.shape)

# Let's examine the first few elements
print("First 10 elements of bands:", bands[:15])

# If you know the expected dimensions of your data, you can reshape it
# For example, if you expect a 3D array with shape (height, width, num_bands):
# Assuming you know the height, width, and number of bands:
# height = ...
# width = ...
# num_bands = ...
# reshaped_bands = bands.reshape(height, width, num_bands)

# If you don't know the exact dimensions, we need more information about your data

Bands shape: (4976640032,)
First 10 elements of bands: [2.2366853e+08 1.2387478e-40 3.4833497e-15 4.4898648e+21 1.5767864e-19
 1.4274554e-08 2.2228396e-15 7.6830766e+31 1.7177136e+19 6.7425655e+22
 1.5767864e-19 1.8727951e+31 2.2228527e-15 2.7904159e+29 1.5767847e-19]


In [11]:
total_elements = bands.shape[0]  # 4976640032
elements_per_image = 2400 * 2400  # 5760000
num_spectral_bands = total_elements // (144 * elements_per_image)

print(f"Number of spectral bands: {num_spectral_bands}")
print(f"Remainder: {total_elements % (144 * elements_per_image)}")

Number of spectral bands: 6
Remainder: 32


In [14]:
total_elements = bands.shape[0]  # 4976640032
elements_per_image = 2400 * 2400  # 5760000
num_time_steps = 144

print(f"Total elements: {total_elements}")
print(f"Elements per image: {elements_per_image}")
print(f"Number of time steps: {num_time_steps}")

# Calculate number of bands
num_bands = total_elements // (num_time_steps * elements_per_image)
print(f"Calculated number of bands: {num_bands}")

# Check if there's any remainder
remainder = total_elements % (num_time_steps * elements_per_image)
print(f"Remainder: {remainder}")

Total elements: 4976640032
Elements per image: 5760000
Number of time steps: 144
Calculated number of bands: 6
Remainder: 32


In [15]:
# Reshape, excluding the extra elements
reshaped_bands = bands[:-32].reshape(144, 6, 2400, 2400)
print(f"Reshaped bands shape: {reshaped_bands.shape}")

Reshaped bands shape: (144, 6, 2400, 2400)


In [16]:
print("Extra elements:")
print(bands[-32:])

Extra elements:
[1.4009205e-29 3.3328318e-32 9.6983885e-27 7.8278344e-28 6.8810742e-28
 3.1310949e-27 1.4009205e-29 3.3328318e-32 2.1820158e-26 5.5559073e-28
 5.9975702e-28 2.1718491e-27 1.4009241e-29 3.8913580e-32 9.6983885e-27
 6.1869999e-28 5.9975702e-28 2.8281813e-27 1.4009241e-29 3.8913580e-32
 9.6983885e-27 7.0705337e-28 5.4611280e-28 4.0407566e-27 1.4009469e-29
 4.1032047e-32 9.6983885e-27 6.0607686e-28 5.4611280e-28 3.5358857e-27
 1.4009469e-29 4.1032047e-32]


In [17]:
for i in range(6):
    band_data = reshaped_bands[:, i, :, :]
    print(f"Band {i+1}:")
    print(f"  Min: {band_data.min()}")
    print(f"  Max: {band_data.max()}")
    print(f"  Mean: {band_data.mean()}")
    print(f"  Std Dev: {band_data.std()}")
    print()

Band 1:
  Min: 0.0
  Max: 1.330661074788474e+37
  Mean: 1.6043001684939487e+28
  Std Dev: inf

Band 2:
  Min: 0.0
  Max: 1.2386726483853312e+16
  Mean: 77204496.0
  Std Dev: 950988374016.0

Band 3:
  Min: 0.0
  Max: 8256743800832.0
  Mean: 80607.1328125
  Std Dev: 573389504.0

Band 4:
  Min: 0.0
  Max: 206472167424.0
  Mean: 1428.552490234375
  Std Dev: 10009167.0

Band 5:
  Min: 0.0
  Max: 19364913152.0
  Mean: 363.0224304199219
  Std Dev: 1473523.0

Band 6:
  Min: 0.0
  Max: 32986546307072.0
  Mean: 39864.7265625
  Std Dev: 1145366272.0



In [32]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Calculate the number of spectral bands
total_elements = bands.shape[0]
elements_per_image = 2400 * 2400
num_spectral_bands = 6  # As calculated

print(f"Number of spectral bands: {num_spectral_bands}")

# Reshape bands to match other arrays, ignoring the extra 32 elements
bands_reshaped = bands[:-32].reshape(144, 2400, 2400, num_spectral_bands)

# Define chunk size (adjust as needed based on your available memory)
chunk_size = 10  # Process 10 time steps at a time

# Get the total number of time steps
total_time_steps = 144

# Initialize an empty list to store processed features
processed_features = []

# Process data in chunks
for start in range(0, total_time_steps, chunk_size):
    end = min(start + chunk_size, total_time_steps)
    
    # Load a chunk of data
    bands_chunk = bands_reshaped[start:end]
    clp_chunk = clp[start:end, ..., 0]  # Remove last dimension
    is_data_chunk = is_data[start:end, ..., 0]  # Remove last dimension
    scl_chunk = scl[start:end, ..., 0]  # Remove last dimension
    norm_factors_chunk = norm_factors[start:end]

    # 1. Apply cloud mask
    cloud_mask = (clp_chunk == 0)  # Assuming 0 means no clouds

    # 2. Apply scene classification mask (keep only vegetation)
    vegetation_mask = np.isin(scl_chunk, [4, 5, 6, 7])  # Assuming these values represent vegetation

    # 3. Combine masks
    valid_pixels = cloud_mask & vegetation_mask & is_data_chunk

    # 4. Apply normalization factors
    normalized_bands = bands_chunk * norm_factors_chunk[:, np.newaxis, np.newaxis, :]

    # 5. Calculate vegetation indices
    # Adjust these indices based on your actual band order
    blue_index, green_index, red_index, nir_index = 0, 1, 2, 3  # Example indices, adjust as needed

    def calculate_ndvi(nir, red):
        return np.where((nir + red) != 0, (nir - red) / (nir + red), 0)

    def calculate_evi(blue, red, nir):
        return np.where((nir + 6 * red - 7.5 * blue + 1) != 0, 
                        2.5 * ((nir - red) / (nir + 6 * red - 7.5 * blue + 1)), 0)

    ndvi = calculate_ndvi(normalized_bands[..., nir_index], normalized_bands[..., red_index])
    evi = calculate_evi(normalized_bands[..., blue_index], normalized_bands[..., red_index], normalized_bands[..., nir_index])

    # 6. Combine all features
    features = np.concatenate([normalized_bands, 
                               ndvi[..., np.newaxis], 
                               evi[..., np.newaxis]], axis=-1)

    # 7. Apply masks to features
    masked_features = features[valid_pixels]

    # Append to processed features
    processed_features.append(masked_features)

    print(f"Processed time steps {start} to {end}")

# Combine all processed features
all_features = np.vstack(processed_features)

# Normalize features to [0, 1] range
scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(all_features)

np.savez_compressed("normalized_features.npz", image_stack=normalized_features)
print("Preprocessed features shape:", normalized_features.shape)

Number of spectral bands: 6
Processed time steps 0 to 10
Processed time steps 10 to 20
Processed time steps 20 to 30
Processed time steps 30 to 40
Processed time steps 40 to 50
Processed time steps 50 to 60
Processed time steps 60 to 70
Processed time steps 70 to 80
Processed time steps 80 to 90
Processed time steps 90 to 100
Processed time steps 100 to 110
Processed time steps 110 to 120
Processed time steps 120 to 130
Processed time steps 130 to 140
Processed time steps 140 to 144
Preprocessed features shape: (168800, 8)


In [27]:
print("Shape of normalized_features:", normalized_features.shape)

Shape of normalized_features: (168800, 8)


In [30]:
print("Dimensions of labels:", labels.shape)
print("Dimensions of normalized_features:", normalized_features.shape)
print("Boolean mask:", crop_mask)
print("Sum of true values in mask:", crop_mask.sum())  # This tells you how many entries match the crop_id


Dimensions of labels: (2064, 6)
Dimensions of normalized_features: (168800, 8)
Boolean mask: 0        True
1       False
2       False
3       False
4       False
        ...  
2059    False
2060    False
2061     True
2062     True
2063    False
Name: crop_id, Length: 2064, dtype: bool
Sum of true values in mask: 742


In [45]:
print("Shape of normalized_features:", normalized_features.shape)
print("Shape of labels:", labels.shape)
print("Number of unique crop_ids:", len(labels['crop_id'].unique()))

Shape of normalized_features: (0, 8)
Shape of labels: (2064, 6)
Number of unique crop_ids: 9
